# Notebook for Capstone project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [21]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
pcdf = tables[0]
pcdf.head()

Postal code           Borough                Neighborhood
0         M1A      Not assigned                         NaN
1         M2A      Not assigned                         NaN
2         M3A        North York                   Parkwoods
3         M4A        North York            Victoria Village
4         M5A  Downtown Toronto  Regent Park / Harbourfront

In [22]:
pcdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [23]:
print(pcdf.shape,  (pcdf["Borough"] == "Not assigned").sum())

(180, 3) 77


In [24]:
pcdf.drop(pcdf[pcdf["Borough"] == "Not assigned"].index, axis=0, inplace=True)
pcdf.reset_index()
print(pcdf.shape,  (pcdf["Borough"] == "Not assigned").sum())

(103, 3) 0


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [25]:
pcdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postal code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [26]:
pcdf.shape

(103, 3)

### Get the latitude and the longitude coordinates of each neighborhood

In [27]:
geocodes = pd.read_csv("http://cocl.us/Geospatial_data")
geocodes.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [28]:
geocodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [33]:
pcdf1 = pd.merge(pcdf, geocodes, left_on='Postal code', right_on='Postal Code', how='left', sort=False).drop("Postal Code", axis=1);

pcdf1.head()

Postal code           Borough                                  Neighborhood  \
0         M3A        North York                                     Parkwoods   
1         M4A        North York                              Victoria Village   
2         M5A  Downtown Toronto                    Regent Park / Harbourfront   
3         M6A        North York             Lawrence Manor / Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [34]:
pcdf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postal code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


### Explore and cluster the neighborhoods in Toronto

#### Create a map of Toranto with neighborhoods superimposed on top.

In [36]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[pcdf1["Latitude"].mean() , pcdf1["Longitude"].mean()], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(pcdf1['Latitude'], pcdf1['Longitude'], pcdf1['Borough'], pcdf1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [37]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

CLIENT_ID = 'BDU3L12SS0PUDVK2QHMIJ1T1MGLJLAQ33HO4OFKXGC0TRNSH' # your Foursquare ID
CLIENT_SECRET = 'AGRGLAMKHXZDYNARONPLJ3RWESASTEF1F3EXIEBWHTZ2RCVZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30



#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [39]:
toronto_venues = getNearbyVenues(names=pcdf1['Neighborhood'],
                                   latitudes=pcdf1['Latitude'],
                                   longitudes=pcdf1['Longitude']
                                  )


toronto_venues.head (20)


Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0                    Parkwoods              43.753259              -79.329656   
1                    Parkwoods              43.753259              -79.329656   
2                    Parkwoods              43.753259              -79.329656   
3             Victoria Village              43.725882              -79.315572   
4             Victoria Village              43.725882              -79.315572   
5             Victoria Village              43.725882              -79.315572   
6             Victoria Village              43.725882              -79.315572   
7             Victoria Village              43.725882              -79.315572   
8   Regent Park / Harbourfront              43.654260              -79.360636   
9   Regent Park / Harbourfront              43.654260              -79.360636   
10  Regent Park / Harbourfront              43.654260              -79.360636   
11  Regent Park / Harbourfront              43.654260              -79.360636   
12  Regent Park / Harbourfront              43.654260              -79.360636   
13  Regent Park / Harbourfront              43.654260              -79.360636   
14  Regent Park / Harbourfront              43.654260              -79.360636   
15  Regent Park / Harbourfront              43.654260              -79.360636   
16  Regent Park / Harbourfront              43.654260              -79.360636   
17  Regent Park / Harbourfront              43.654260              -79.360636   
18  Regent Park / Harbourfront              43.654260              -79.360636   
19  Regent Park / Harbourfront              43.654260              -79.360636   

                                        Venue  Venue Latitude  \
0                             Brookbanks Park       43.751976   
1                               Variety Store       43.751974   
2                  TTC stop - 44 Valley Woods       43.755402   
3                      Victoria Village Arena       43.723481   
4                                 Tim Hortons       43.725517   
5                                   Portugril       43.725819   
6   Eglinton Ave E & Sloane Ave/Bermondsey Rd       43.726086   
7                                  Pizza Nova       43.725824   
8                            Roselle Desserts       43.653447   
9                               Tandem Coffee       43.653559   
10                     Cooper Koo Family YMCA       43.653249   
11                        Body Blitz Spa East       43.654735   
12                         Morning Glory Cafe       43.653947   
13                             Impact Kitchen       43.656369   
14                            Corktown Common       43.655618   
15                     Figs Breakfast & Lunch       43.655675   
16           The Distillery Historic District       43.650244   
17                   Dominion Pub and Kitchen       43.656919   
18                   Distillery Sunday Market       43.650075   
19                        SOMA chocolatemaker       43.650622   

    Venue Longitude         Venue Category  
0        -79.332140                   Park  
1        -79.333114      Food & Drink Shop  
2        -79.333741               Bus Stop  
3        -79.315635           Hockey Arena  
4        -79.313103            Coffee Shop  
5        -79.312785  Portuguese Restaurant  
6        -79.313620           Intersection  
7        -79.312860            Pizza Place  
8        -79.362017                 Bakery  
9        -79.361809            Coffee Shop  
10       -79.358008    Distribution Center  
11       -79.359874                    Spa  
12       -79.361149         Breakfast Spot  
13       -79.356980             Restaurant  
14       -79.356211                   Park  
15       -79.364503         Breakfast Spot  
16       -79.359323          Historic Site  
17       -79.358967                    Pub  
18       -79.361832         Farmers Market  
19       -79.358127         Chocolate Shop

In [40]:
toronto_venues.shape

(1337, 7)

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#manhattan_onehot.groupby('Neighborhood').Name.apply(lambda n: n + np.concatenate(([''], (np.arange(len(n))+1).astype(str)[1:])))
toronto_onehot.head()


Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0             0  ...      0              0                              0   
1             0  ...      0              0                              0   
2             0  ...      0              0                              0   
3             0  ...      0              0                              0   
4             0  ...      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                 0                      0                0         0   
1                 0                      0                0         0   
2                 0                      0                0         0   
3                 0                      0                0         0   
4                 0                      0                0         0   

   Wine Shop  Wings Joint  Women's Store  
0          0            0              0  
1          0            0              0  
2          0            0              0  
3          0            0              0  
4          0            0              0  

[5 rows x 230 columns]

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                           Agincourt          0.0   
1                             Alderwood / Long Branch          0.0   
2   Bathurst Manor / Wilson Heights / Downsview North          0.0   
3                                     Bayview Village          0.0   
4                  Bedford Park / Lawrence Manor East          0.0   
..                                                ...          ...   
88                                 Wexford / Maryvale          0.0   
89                                         Willowdale          0.0   
90                                             Woburn          0.0   
91                                   Woodbine Heights          0.0   
92                                    York Mills West          0.0   

    Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                 0.0      0.0                 0.0           0.0   
1                 0.0      0.0                 0.0           0.0   
2                 0.0      0.0                 0.0           0.0   
3                 0.0      0.0                 0.0           0.0   
4                 0.0      0.0                 0.0           0.0   
..                ...      ...                 ...           ...   
88                0.0      0.0                 0.0           0.0   
89                0.0      0.0                 0.0           0.0   
90                0.0      0.0                 0.0           0.0   
91                0.0      0.0                 0.0           0.0   
92                0.0      0.0                 0.0           0.0   

    Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0              0.0              0.0               0.0             0.000000   
1              0.0              0.0               0.0             0.000000   
2              0.0              0.0               0.0             0.000000   
3              0.0              0.0               0.0             0.000000   
4              0.0              0.0               0.0             0.043478   
..             ...              ...               ...                  ...   
88             0.0              0.0               0.0             0.000000   
89             0.0              0.0               0.0             0.000000   
90             0.0              0.0               0.0             0.000000   
91             0.0              0.0               0.0             0.000000   
92             0.0              0.0               0.0             0.000000   

    ...  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0   ...    0.0            0.0                            0.0   
1   ...    0.0            0.0                            0.0   
2   ...    0.0            0.0                            0.0   
3   ...    0.0            0.0                            0.0   
4   ...    0.0            0.0                            0.0   
..  ...    ...            ...                            ...   
88  ...    0.0            0.0                            0.0   
89  ...    0.0            0.0                            0.0   
90  ...    0.0            0.0                            0.0   
91  ...    0.0            0.0                            0.0   
92  ...    0.0            0.0                            0.0   

    Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                0.0               0.000000              0.0       0.0   
1                0.0               0.000000              0.0       0.0   
2                0.0               0.000000              0.0       0.0   
3                0.0               0.000000              0.0       0.0   
4                0.0               0.000000              0.0       0.0   
..               ...                    ...              ...       ...   
88               0.0               0.000000              0.0       0.0   
89               0.0               0.027027              0.0       0.0   
90         

### Cluster Neighborhoods

In [44]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1])

In [51]:
kmeans.labels_.shape, pcdf1.shape

((93,), (103, 5))

In [57]:
toronto_merged = toronto_grouped

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(toronto_merged, pcdf1, left_on='Neighborhood', right_on='Neighborhood', how='left', sort=False)

toronto_merged.head() # check the last columns!

Neighborhood  Yoga Studio  \
0                                          Agincourt          0.0   
1                            Alderwood / Long Branch          0.0   
2  Bathurst Manor / Wilson Heights / Downsview North          0.0   
3                                    Bayview Village          0.0   
4                 Bedford Park / Lawrence Manor East          0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.043478   

   ...  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  Women's Store  \
0  ...              0.0       0.0        0.0          0.0            0.0   
1  ...              0.0       0.0        0.0          0.0            0.0   
2  ...              0.0       0.0        0.0          0.0            0.0   
3  ...              0.0       0.0        0.0          0.0            0.0   
4  ...              0.0       0.0        0.0          0.0            0.0   

   Cluster Labels  Postal code      Borough   Latitude  Longitude  
0               1          M1S  Scarborough  43.794200 -79.262029  
1               4          M8W    Etobicoke  43.602414 -79.543484  
2               1          M3H   North York  43.754328 -79.442259  
3               1          M2K   North York  43.786947 -79.385975  
4               1          M5M   North York  43.733283 -79.419750  

[5 rows x 235 columns]

### Display neighborhood clusters in map

In [58]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[pcdf1["Latitude"].mean() , pcdf1["Longitude"].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters